In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
working_path = os.getcwd()

In [ ]:
import tensorflow as tf
import numpy as np
import ag.logging as log
from PIL import Image
log.set(5)

In [ ]:
load = np.load("super_set.npz")
imgs = load['images']
labels = load['labels']
dataset_examples = imgs.shape[0]
dataset_h = imgs.shape[1]
dataset_w = imgs.shape[2]
dataset_c = imgs.shape[3]
dataset_classes = labels.shape[1]
dataset_shape = imgs[0].shape
label_example = labels[0]

In [ ]:
log.info("Images: {}".format(imgs.shape))
log.info("Labels: {}".format(labels.shape))
log.info("Image shape: {}".format(dataset_shape))
log.info("Label: {}".format(label_example))

In [ ]:
with tf.name_scope("InitVars"):
    with tf.variable_scope("init_vars_values"):
        global_step = tf.Variable(name='global_step',
                                  trainable=False,
                                  initial_value=0,
                                  collections=["echo", "step"])
        
        learn_rate = tf.train.exponential_decay(0.1, global_step,
                                                .05, 0.87,
                                                staircase=True, name="Learn_decay")
        
tf.add_to_collection("learn_rate", learn_rate)
tf.summary.scalar("learn_rate", learn_rate)
tf.summary.scalar("global_step", learn_rate)
tf.summary.histogram('decay_rate', learn_rate)

In [ ]:
with tf.variable_scope("inputs"):
    Input_Tensor_Image = tf.placeholder(tf.float32, [None, dataset_h, dataset_w,
                                                           dataset_c], name="Input_Tensor")
tf.add_to_collection("input_tensor", Input_Tensor_Image)

In [ ]:
Input_Tensor_Labels = tf.placeholder(tf.float32, [None, dataset_classes], name="Input_Label")
tf.add_to_collection("label", Input_Tensor_Labels)
Input_True_Labels = tf.argmax(Input_Tensor_Labels, dimension=1)

In [ ]:
with tf.variable_scope("keep_prob"):
    keep_prob = tf.placeholder(tf.float32, name="Keep_prob")  # new feature goes with the dropout option
    tf.add_to_collection("keep_prob", keep_prob)

In [ ]:
def BUILD_CONV_LAYERS(x_image, layers, f_size=5):
    """
    Magically construct many Convolutional layers for a Neural Network.
    :return:
        An unused list of all the layers_names, weights and biases for each layer. these are added to tensorboard
        automatically.
    """
    start_shape = x_image
    log.debug("Start shape= {}".format(start_shape))
    reducing_shape = 0
    last_num_f = 0
    log.info("Building {} Convolutional Layers".format(layers))
    for layer in range(layers):
        num_f = 16 * (layer + 1)
        log.debug("Current Num of Features= {}".format(num_f))
        if layer == 0:
            img = start_shape
            channel = dataset_c
        elif layer > 0:
            img = reducing_shape
            channel = last_num_f

        last_num_f = num_f
        log.debug("Start shape= {}".format(start_shape))
        debugs = "New Features = {}".format(channel)
        reducing_shape, w, b = new_conv_layer(input=img,
                                              filter_size=f_size,
                                              chan=channel,
                                              num_filters=num_f)
        layer_name = "convLayer_%s" % layer
        log.debug("Finishing Layer {}\n\t{}".format(layer_name, debugs))
        tf.summary.histogram("weights{}".format(layer), w)
        tf.summary.histogram("biases{}".format(layer), b)
        log.info("Finished Layer {}\n\t{}".format(layer_name, debugs))

    log.info("Finished Building {} Conv Layers\nMoving To Flattening...".format(layers))
    return reducing_shape, last_num_f

In [ ]:
def BUILD_FC_LAYER(x_image, in_features, num_fc, num_final):
    """
    Magically create a huge number of Fully connected layers.
    :param layers:
        Number of FC layers to create.
    :return:
        returns a human readable non used list of the layer names. These are added to tensorboard automatically.
    """
    log.info("Building {} Fully Connected Layers\ninput features size = {}\n*-".format(num_fc, in_features))
    # first time thru options
    inputs = in_features
    features = 0
    use_reLu = True
    use_Drop = True
    current_layer = False    
    for layer in range(num_fc):        
        if layer == 0:  # if first time through
            current_layer = x_image
            
        else:
            inputs = features
        
        features = int(inputs / (layer + 1)) # avoid a 0 zero divion error..
        
        if layer == num_fc - 1:  # if last time through
            features = num_final
            use_reLu = False  # dont use on the last time thru
            use_Drop = False
        
        log.debug("Layer-{}\n\t# inputs: {}\n\t# outputs: {}\n\tuse relu? {}\n\tuse drop? {}".format(\
            current_layer, inputs, features, use_reLu, use_Drop))
        current_layer, w, b = new_fc_layer(input_=current_layer,
                                           num_inputs=inputs,
                                           num_outputs=features,
                                           use_relu=use_reLu,
                                           use_drop=use_Drop)
             
        layer_name = "fullyLayer_{}".format(layer)
        with tf.name_scope(layer_name):
            with tf.name_scope("weights"):
                tf.summary.histogram("weights", w)
            with tf.name_scope("biases"):
                tf.summary.histogram('biases', b)
        log.info("{} : {}".format(layer_name, current_layer))
    log.info("Finished Building {} Fully Connected Layers".format(num_fc))
    return current_layer # final layer

In [ ]:
def new_fc_layer(input_, num_inputs, num_outputs, use_relu=True, use_drop=False):
    weights = new_weights(shape=[num_inputs, num_outputs])  # set weights
    biases = new_biases(length=num_outputs)  # set number of OUTPUTS like give me top k or whatever...
    layer = tf.matmul(input_, weights) + biases  # this is a #BIGMATH func
    if use_relu:
        layer = tf.nn.relu(layer)
    if use_drop:
        layer = tf.nn.dropout(layer, keep_prob)
    return layer, weights, biases

In [ ]:
def new_weights(shape):
    """This generates a new weight for each layer"""
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name="weight", collections=["weights"])

def new_biases(length):
    """This generates a new bias for each layer"""
    return tf.Variable(tf.constant(0.1, shape=[length]), name="bias", collections=["biases"])

In [ ]:
def new_conv_layer(input, filter_size, chan, num_filters, use_pooling=True):
    log.debug("Building a conv layer")
    X_shape = [filter_size, filter_size, chan, num_filters]
    log.debug("current shape: {}".format(X_shape))
    weights = new_weights(shape=X_shape)
    log.debug("weights shape = {}".format(weights))
    biases = new_biases(length=num_filters)
    log.debug("weights shape = {}".format(biases))
    """ THis is the MAGIC again... """
    layer = tf.nn.conv2d(input=input,  # This is the output of the last layer
                         filter=weights,  # this is a thing
                         strides=[1, 1, 1, 1],  # 1111 is NOT pooled MAX work
                         padding='SAME')  # input output transformation
    layer += biases  # Add biases
    if use_pooling:  # this skips pixels... saves time but skips things obviously
        log.debug("using pooling")
        layer = tf.nn.max_pool(value=layer,  # take the weights and bias together as an input
                               ksize=[1, 2, 2, 1],  # stuff
                               strides=[1, 2, 2, 1],  # 2 x 2 stride... could increase... check
                               padding='SAME')  # i feel like this should already be in variable, but w/e
    layer = tf.nn.relu(layer)  # rectified linear Unit ... like a boss
    log.debug("Finished Building a conv Layer:\n\t{}".format(layer))
    return layer, weights, biases

In [ ]:
def flatten_layer(layer):
    log.info("Tranitioning from Conv to fc layers")
    layer_shape = layer.get_shape()  # ASSERT layer_shape == [num_images, img_height, img_width, num_channels]
    num_features = layer_shape[1:4].num_elements()  # like a boss...
    log.debug("Shape: {}, Features: {}".format(layer_shape, num_features))
    layer_flat = tf.reshape(layer, [-1, num_features])  # yep...
    log.info("Finished Flattening Layers")
    return layer_flat, num_features

In [ ]:
with tf.variable_scope("conv_layers"):
    x_image, num_features = BUILD_CONV_LAYERS(Input_Tensor_Image, 2)
log.info("created {} layers with {} features".format(2, num_features))

In [ ]:
with tf.variable_scope("Flat_layer"):
    x_image, features =flatten_layer(x_image)

In [ ]:
with tf.variable_scope("FC_layers"):
    final_layer = BUILD_FC_LAYER(x_image, features, num_fc=5, num_final=5)  # build FC LAYERS

In [ ]:
with tf.variable_scope("sophmax"):
    sophmax = tf.nn.softmax(final_layer, name="sophmax")
    tf.add_to_collection("sophmax_layer", sophmax)
tf.summary.histogram('activations', sophmax)

In [ ]:
### TRAINING METHOD FROM MUPEN #### MEPHOD?
with tf.variable_scope("mupen_method"):
    cost = tf.reduce_mean(tf.square(tf.subtract(Input_Tensor_Labels, final_layer)))
    training_vars = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()]) * 0.001
    loss = cost + training_vars
    tf.add_to_collection("loss", loss)
tf.summary.scalar("train_cost", loss)

In [ ]:
with tf.variable_scope('Dropout_Optimizer_Train'):
     train_drop_loss = tf.train.AdamOptimizer(learn_rate).minimize(loss, global_step=global_step)
     tf.add_to_collection("train_op", train_drop_loss)

In [ ]:
### OHHHH WE ARE GETTING CLOSE! ###
""" Initialize the session """
init_op = tf.global_variables_initializer

"""create summary op"""
merged = tf.summary.merge_all

"""create saver object"""
saver = tf.train.Saver(
            #var_list={"{}".format(v): v for v in [tf.model_variables()]},
            write_version=tf.train.SaverDef.V2,
            sharded=True,
            keep_checkpoint_every_n_hours=.001
            )

In [ ]:
### Session Manger for distrubted train... SOON!!!###
#self.bossMan = tf.train.Supervisor(is_chief=True,
#                                   logdir='~/notebooks/logdir/',
#                                   checkpoint_basename='check.point',
#                                   init_op = init_op,
#                                   summary_op = merged,
#                                   saver = saver,
#                                   global_step = global_step,
#                                   save_model_secs = 60
#                                   )

In [ ]:
### last thing we need is a batch mechiniism...
def get_batch(): pass                                                                                                                                             

In [ ]:
# Single use for demonstration Session...
# Start session
sess = tf.InteractiveSession()
log.info("Started Session: {}".format(sess))
sess.run(init_op())
log.info("started Init Op")

In [ ]:
# Training loop variables
epochs = 100
batch_size = 50
num_samples = imgs.shape[0]
step_size = int(num_samples / batch_size) 
log.info("Training Run Options:\n\tTraining data resets: {}\n\tBatch size: {}\n\tStep Size: {}\n\t".format(\
            epochs, batch_size, step_size))

In [ ]:
### Training Proceedures
current_epoch = 0

while current_epoch < epochs:
    log.info("Currently on epoch: {}".format(current_epoch))
    for i in range(step_size):
        feed_dict = {Input_Tensor_Images: batch[0],
                     Input_Tensor_Labels: batch[1],
                     keep_prob: 0.8}

        _, current_step, summary, loss, learn = sess.run([train_drop_loss,
                                                          merged,
                                                          learn_rate], 
                                                         feed_dict)

    log.info("\t-Current Step: {}\n\t-Loss: {}\n\t-Learn rate: {}".format(current_step, loss, learn))

print("finished training")

In [ ]:
log.info("Starting the Ipython Training!")
sess = tf.Session()
log.debug("session started")
#init graph, load model
sess.run(init_op())
train_writer = tf.summary.FileWriter(save_path, sess.graph)
# test_writer = tf.summary.FileWriter(self.options.logDir, sess.graph)
log.debug("init started")
offset = 0
iters = 1  # testing
log.info("Training for {} iters".format(iters))
for step in range(iters):
    feed_dict={input_holder: batch_data, input_label: batch_labels}
    _, current_step, summary, cost, learn = sess.run([train_op,
                                                      global_step,
                                                      merged,
                                                      cost,
                                                      learn_rate], 
                                                     feed_dict)

    # _, training_loss = sess.run([train_op, cost], feed_dict=feed_dict)

    if step % log_every == 0:
        # print('training loss at step %d: %.2f (%s)' % (step, training_loss, datetime.datetime.now()))
        train_writer.add_summary(summary, step)
        log.info("{} :: Current Step: {}\n\t-Loss: {}\n\t-Learn rate: {}\n{}\n".format(datetime.time(), current_step, cost, learn, buf))

        if step % test_every == 0:
            # creating test
            prediction = test_op.eval({test_data: test_X})

            print(buf)
            print("test_loss: {}".format(prediction))
            print(buf)

            # saver.save(sess, save_path, global_step=step)
saver.save(sess, save_path, global_step=step)